In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm import tqdm
import numpy as np
from numpy import linalg as LA
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

import json
import keras
import tensorflow
from keras import layers, Model
from keras.models import Sequential
from keras.applications import DenseNet201
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, roc_curve

In [ ]:
csv_path = 'drive/My Drive/Team No Name/UrbanSound8K/metadata/UrbanSound8K.csv'
spectrograms_path = "drive/My Drive/Team No Name/numpySpectrograms/"
model_save_path = "drive/My Drive/Team No Name/basemodel-known"
embeddings_path = "drive/My Drive/Team No Name/embeddings/"
embeddings_model_save_path = "drive/My Drive/Team No Name/feature_extraction_test"
test_size = 0.2
val_size = 0.2
batch_size = 16
num_classes = 5

In [ ]:
class SpecLoader(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, spec_dir):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.spec_dir = spec_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs), np.array(batch_y)

In [ ]:
data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < 5]
data_df_unknown = data_df.loc[data_df["classID"] >= 5]

X_trainval, known_X_test, y_trainval, known_y_test = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=test_size, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=val_size, random_state = 42)
X_trash, X_unknown, y_trash, y_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=test_size, random_state = 42)
y_unknown = [-1] * len(y_unknown)

X_test = known_X_test + X_unknown
y_test = known_y_test + y_unknown
train_loader = SpecLoader(X_train, y_train, batch_size, spectrograms_path)
known_test_loader = SpecLoader(known_X_test, known_y_test, batch_size, spectrograms_path)
test_loader = SpecLoader(X_test, y_test, batch_size, spectrograms_path)
val_loader = SpecLoader(X_val, y_val, batch_size, spectrograms_path)
trainval_loader = SpecLoader(X_trainval, y_trainval, batch_size, spectrograms_path)

# Base Model

In [ ]:
densenet = DenseNet201(
            include_top=False,
            weights="imagenet",
            input_tensor=None,
            input_shape=None,
            pooling=None)
model = Sequential()
model.add(densenet)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(num_classes, activation="softmax"))
model.summary()
model.compile(
      optimizer="Adam",
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"])

74842112/74836368 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, None, None, 1920)  18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 9605      
Total params: 18,331,589
Trainable params: 18,102,533
Non-trainable params: 229,056
_________________________________________________________________


In [ ]:
earlystopping = EarlyStopping(
                    patience=5, 
                    restore_best_weights=True)
checkpoint = ModelCheckpoint(
                    model_save_path, 
                    monitor="val_accuracy", 
                    save_best_only=True)

model.fit(x=train_loader,
          validation_data=val_loader,
          callbacks=[checkpoint, earlystopping],
          epochs=70,
          verbose=1
         )

Epoch 1/70
178/178 [==============================] - 180s 1s/step - loss: 0.7785 - accuracy: 0.7382 - val_loss: 15.3976 - val_accuracy: 0.4118
Epoch 2/70
178/178 [==============================] - 50s 283ms/step - loss: 0.4679 - accuracy: 0.8483 - val_loss: 0.1058 - val_accuracy: 0.8688
Epoch 3/70
178/178 [==============================] - 50s 283ms/step - loss: 0.3411 - accuracy: 0.8800 - val_loss: 0.0637 - val_accuracy: 0.8858
Epoch 4/70
178/178 [==============================] - 51s 285ms/step - loss: 0.2386 - accuracy: 0.9143 - val_loss: 0.3492 - val_accuracy: 0.8632
Epoch 5/70
178/178 [==============================] - 50s 282ms/step - loss: 0.2107 - accuracy: 0.9255 - val_loss: 0.0042 - val_accuracy: 0.9055
Epoch 6/70
178/178 [==============================] - 50s 283ms/step - loss: 0.1638 - accuracy: 0.9425 - val_loss: 0.0363 - val_accuracy: 0.8463
Epoch 7/70
178/178 [==============================] - 50s 282ms/step - loss: 0.1407 - accuracy: 0.9499 - val_loss: 0.0743 - val_acc

In [ ]:
model.load_weights(model_save_path)
# test_loss, test_accuracy= model.evaluate(x=known_test_loader)
# print(test_loss, test_accuracy)

# Softmax Threshold

In [ ]:
def generate_thresholds(softmaxes, predictions, expected):
    trainval_df = pd.DataFrame([np.amax(softmaxes, 1), predictions, expected])
    trainval_df = trainval_df.transpose()
    trainval_df.columns = ["probability", "predicted", "expected"]
    thresholds = []
    for i in range(num_classes):
        class_df = trainval_df.loc[trainval_df["expected"] == i]
        class_df = class_df.loc[class_df["expected"] == class_df["predicted"]]
        thresholds.append(class_df["probability"].min())
    return thresholds

def check_negative(sample_softmax, thresholds):
    probability = max(sample_softmax)
    idx = sample_softmax.index(probability)
    if probability < thresholds[idx]:
        return -1
    return idx

def evaluate(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))

In [ ]:
trainval_softmaxes = model.predict(x=trainval_loader)
trainval_predictions = model.predict_classes(x=trainval_loader, batch_size=None)
thresholds = generate_thresholds(trainval_softmaxes, trainval_predictions, y_trainval)

test_softmaxes = model.predict(x=test_loader)
test_class = []
for softmax in test_softmaxes:
    test_class.append(check_negative(softmax.tolist(), thresholds))
    
evaluate(test_class, y_test)

Overall accuracy: 0.5054378935317687
Accuracy for class 2: 0.9310344827586207
Accuracy for class 4: 0.9651162790697675
Accuracy for class 0: 0.9808612440191388
Accuracy for class 3: 0.8947368421052632
Accuracy for class 1: 0.972972972972973
Accuracy for class -1: 0.0545876887340302


In [ ]:
# Macro F1-score will give the same importance to each label/class. It will be low for models that only perform well on the common classes while performing poorly on the rare classes.
macro_f1_score = f1_score(test_class, y_test, average='macro')
print("Macro F1 score: {}".format(macro_f1_score))

# Micro-averaging will put more emphasis on the common labels in the data set.
micro_f1_score = f1_score(test_class, y_test, average='micro')
print("Micro F1 score: {}".format(micro_f1_score))

# This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.
weighted_f1_score = f1_score(test_class, y_test, average='weighted')
print("Weighted F1 score: {}".format(weighted_f1_score))

# Scores for each class are returned.
norm_f1_score = f1_score(test_class, y_test, average=None)
print("F1 score for each class: {}".format(norm_f1_score))

Macro F1 score: 0.5644870096815503
Micro F1 score: 0.5054378935317687
Weighted F1 score: 0.6273909324342298
F1 score for each class: [0.10295728 0.6710311  0.68246445 0.76056338 0.62385321 0.54605263]


### Set threshold using ROC curve

In [ ]:
def generate_thresholds_roc(softmaxes, expected):
    thresholds = []
    # create binary class for each class
    for i in range(5):
      binary_expected = [1 if x == i else 0 for x in expected]
      fpr, tpr, threshold = roc_curve(binary_expected, np.amax(softmaxes, 1))
      i = np.arange(len(tpr)) 
      roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
      roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]
      updated_threshold = list(roc_t['threshold'])[0]
      thresholds.append(updated_threshold)
    return thresholds

def check_negative_roc(sample_softmax, thresholds):
    probability = max(sample_softmax)
    idx = sample_softmax.index(probability)
    count = 0
    for t in thresholds:
        if probability < t:
          count += 1
    if count == len(thresholds):
      return -1
    return idx

In [ ]:
roc_thresholds = generate_thresholds_roc(trainval_softmaxes, y_trainval)
print("Optimal threshold set by ROC curve: {}".format(roc_thresholds))

Optimal threshold set by ROC curve: [0.9997814297676086, 0.9998383522033691, 0.9995611310005188, 0.9993612170219421, 0.9996367692947388]


In [ ]:
test_softmaxes = model.predict(x=test_loader)
test_class = []
for softmax in test_softmaxes:
    test_class.append(check_negative_roc(softmax.tolist(), roc_thresholds))
    
evaluate(test_class, y_test)

Overall accuracy: 0.7057813394390383
Accuracy for class 2: 0.4630541871921182
Accuracy for class 4: 0.5116279069767442
Accuracy for class 0: 0.6363636363636364
Accuracy for class 3: 0.33771929824561403
Accuracy for class 1: 0.7432432432432432
Accuracy for class -1: 0.9128919860627178


In [ ]:
trainval_predictions = model.predict_classes(x=trainval_loader, batch_size=None)
evaluate(trainval_predictions, y_trainval)

Overall accuracy: 0.9774202653118825
Accuracy for class 4: 0.9927536231884058
Accuracy for class 0: 0.9949431099873578
Accuracy for class 3: 0.9637305699481865
Accuracy for class 1: 0.9802816901408451
Accuracy for class 2: 0.9560853199498118


In [ ]:
trainval_class = []
for softmax in trainval_softmaxes:
    trainval_class.append(check_negative_roc(softmax.tolist(), roc_thresholds))
    
evaluate(trainval_class, y_trainval)

Overall accuracy: 0.5619531470505221
Accuracy for class 4: 0.5640096618357487
Accuracy for class 0: 0.7243994943109987
Accuracy for class 3: 0.3898963730569948
Accuracy for class 1: 0.7295774647887324
Accuracy for class 2: 0.4905897114178168


In [ ]:
macro_f1_score = f1_score(test_class, y_test, average='macro')
print("Macro F1 score: {}".format(macro_f1_score))

micro_f1_score = f1_score(test_class, y_test, average='micro')
print("Micro F1 score: {}".format(micro_f1_score))

weighted_f1_score = f1_score(test_class, y_test, average='weighted')
print("Weighted F1 score: {}".format(weighted_f1_score))

norm_f1_score = f1_score(test_class, y_test, average=None)
print("F1 score for each class: {}".format(norm_f1_score))

Macro F1 score: 0.6427631320302183
Micro F1 score: 0.6971951917572983
Weighted F1 score: 0.7176721974743813
F1 score for each class: [0.75383899 0.68452381 0.73770492 0.58983051 0.50326797 0.58741259]


# Feature Extractor

### Base Model

In [ ]:
feature_extractor = Model(inputs = model.inputs, outputs = model.layers[-2].output)
feature_extractor.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201_input (InputLaye [(None, None, None, 3)]   0         
_________________________________________________________________
densenet201 (Functional)     (None, None, None, 1920)  18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
Total params: 18,321,984
Trainable params: 18,092,928
Non-trainable params: 229,056
_________________________________________________________________


### VGGish Model

In [ ]:
class EmbeddingsLoader(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, emb_dir):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.emb_dir = emb_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) *
    self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

    batchEmbs = []
    for fileName in batch_x:
        emb = np.load(self.emb_dir + fileName + ".npy")[0]
        batchEmbs.append(emb)
    batchEmbs = np.array(batchEmbs)
    return batchEmbs, np.array(batch_y)

In [ ]:
vggish_train_loader = EmbeddingsLoader(X_train, y_train, batch_size, embeddings_path)
vggish_test_loader = EmbeddingsLoader(X_test, y_test, batch_size, embeddings_path)
vggish_val_loader = EmbeddingsLoader(X_val, y_val, batch_size, embeddings_path)
vggish_trainval_loader = EmbeddingsLoader(X_trainval, y_trainval, batch_size, embeddings_path)

# Similarity to centroid

## Euclidean Distance

In [ ]:
def euclidean_dist(centroid, data_point):
    return LA.norm(centroid - data_point)
    
def generate_centroids(features, expected):
    result_dict = {}
    for i in range(len(features)):
        c = expected[i]
        f = features[i]
        if c not in result_dict:
            result_dict[c] = [np.array(f)]
        else:
            result_dict[c].append(np.array(f))
    for k, v in result_dict.items():
        result_dict[k] = [v, np.mean(np.array(v), axis=0)]
    return result_dict

def generate_thresholds_dist(centroids):
    result_dict = {}
    for k, [points, centroid] in centroids.items():
        current_max = -1
        for point in points:
            current_dist = euclidean_dist(centroid, point)
            if current_max < current_dist:
                current_max = current_dist
        result_dict[k] = current_max
    return result_dict

In [ ]:
def check_negative(threshold, centroid, feature):
    dist = euclidean_dist(centroid, feature)
    print(dist)
    print(threshold)
    print()
    if dist > threshold:
        return True
    return False

### Base Model

In [ ]:
extracted_features = feature_extractor.predict(x = trainval_loader)

centroids_dict = generate_centroids(extracted_features, y_trainval)
thresholds_dict = generate_thresholds_dist(centroids_dict)
print(thresholds_dict)

{4: 27.000278, 0: 19.336025, 3: 54.661667, 1: 31.284636, 2: 19.541674}


In [ ]:
test_features = feature_extractor.predict(x = test_loader)

In [ ]:
test_classes = model.predict_classes(x=test_loader, batch_size=None)

In [ ]:
updated_predictions = []

for i in range(len(test_features)):
    predicted_class = test_classes[i]
    threshold = thresholds_dict[predicted_class]
    centroid = centroids_dict[predicted_class][1]
    feature = test_features[i]
    for c, thresh in thresholds_dict.items():
        print(euclidean_dist(centroids_dict[c][1], feature))
    print()
    check = check_negative(threshold, centroid, feature)
    if check:
        updated_predictions.append(-1)
    else:
        updated_predictions.append(predicted_class)

Streaming output truncated to the last 5000 lines.
16.630543

11.560316
54.661667

13.521412
10.466235
17.102825
22.627865
21.062817

10.466235
19.336025

8.5056
19.674131
16.821896
22.186884
21.665129

8.5056
27.000278

20.338982
19.79765
18.49885
14.87995
9.970603

9.970603
19.541674

12.173965
13.282677
16.526258
18.785336
19.52238

12.173965
27.000278

14.630336
13.417096
16.11815
17.617628
15.089116

13.417096
19.336025

15.043248
15.390046
14.963867
14.164211
14.761058

14.164211
31.284636

10.70269
21.163765
15.051924
18.396797
15.602848

10.70269
27.000278

17.00115
19.324425
17.004105
10.715025
12.919426

10.715025
31.284636

18.315367
20.185625
18.192015
16.14195
8.785779

8.785779
19.541674

10.5086155
18.5901
12.483031
20.729748
14.76169

12.483031
54.661667

11.452063
16.847862
12.59609
20.705847
17.013113

12.59609
54.661667

18.714346
17.956406
16.004002
13.070864
10.561116

10.561116
19.541674

17.548315
13.048171
12.897671
18.790792
17.949215

12.897671
54.661667

11.8

In [ ]:
evaluate(updated_predictions, y_test)

Overall accuracy: 0.4785346307956497
Accuracy for class 2: 0.9359605911330049
Accuracy for class 4: 0.9651162790697675
Accuracy for class 0: 0.9856459330143541
Accuracy for class 3: 0.8903508771929824
Accuracy for class 1: 0.9594594594594594
Accuracy for class -1: 0.0


In [ ]:
macro_f1_score = f1_score(updated_predictions, y_test, average='macro')
print("Macro F1 score: {}".format(macro_f1_score))

micro_f1_score = f1_score(updated_predictions, y_test, average='micro')
print("Micro F1 score: {}".format(micro_f1_score))

weighted_f1_score = f1_score(updated_predictions, y_test, average='weighted')
print("Weighted F1 score: {}".format(weighted_f1_score))

norm_f1_score = f1_score(updated_predictions, y_test, average=None)
print("F1 score for each class: {}".format(norm_f1_score))

Macro F1 score: 0.5357650047244799
Micro F1 score: 0.4785346307956496
Weighted F1 score: 0.6312004428549921
F1 score for each class: [0.         0.66237942 0.6635514  0.73929961 0.60687593 0.54248366]


### Vggish Model

In [ ]:
vggish_trainval_features = None
for data, labels in tqdm(vggish_trainval_loader):
  if vggish_trainval_features is None: vggish_trainval_features = data
  else: vggish_trainval_features = np.concatenate((vggish_trainval_features, data), axis=0)

100%|██████████| 222/222 [17:32<00:00,  4.74s/it]


In [ ]:
vggish_centroids_dict = generate_centroids(vggish_trainval_features, y_trainval)
vggish_thresholds_dict = generate_thresholds_dist(vggish_centroids_dict)
print(vggish_thresholds_dict)

{4: 1127.6519556049934, 0: 1239.9636677483186, 3: 1271.5463307512155, 1: 1109.0767031765936, 2: 1116.1240536816063}


In [ ]:
test_classes = model.predict_classes(x=test_loader, batch_size=None)

In [ ]:
vggish_test_features = None
for data, labels in tqdm(vggish_test_loader):
  if vggish_test_features is None: vggish_test_features = data
  else: vggish_test_features = np.concatenate((vggish_test_features, data), axis=0)

100%|██████████| 110/110 [08:15<00:00,  4.50s/it]


In [ ]:
updated_predictions = []

for i in tqdm(range(len(vggish_test_features))):
    predicted_class = test_classes[i]
    threshold = vggish_thresholds_dict[predicted_class]
    centroid = vggish_centroids_dict[predicted_class][1]
    feature = vggish_test_features[i]
    for c, thresh in vggish_thresholds_dict.items():
        print(euclidean_dist(vggish_centroids_dict[c][1], feature))
    print()
    check = check_negative(threshold, centroid, feature)
    if check:
        updated_predictions.append(-1)
    else:
        updated_predictions.append(predicted_class)

 15%|█▌        | 263/1747 [00:00<00:01, 1368.41it/s]

1046.2610174113559
1103.0231006406839
827.6471262243332
1018.3905728556084
773.1608702670204

773.1608702670204
1116.1240536816063

766.1053604550293
864.4318024447975
872.0836157759818
881.7485307939168
916.9221857249753

766.1053604550293
1127.6519556049934

735.9690681079269
821.1705012678856
810.542322288585
831.2585679368956
852.0688247598399

735.9690681079269
1127.6519556049934

944.7722296035208
925.892425477084
814.9600589426144
935.4494188459377
722.3697712346581

722.3697712346581
1116.1240536816063

685.3740035682749
645.4510685655019
660.8251426620662
735.9020744002061
636.5292247982173

685.3740035682749
1127.6519556049934

858.9725272352915
943.4179941828539
911.0344764940944
840.1330468122077
968.8815946032774

858.9725272352915
1127.6519556049934

903.8860978615073
780.0924535896329
1025.234978122601
958.6075441336286
1026.9875303638535

780.0924535896329
1239.9636677483186

1031.9691173028014
980.1233873101374
838.5758521731444
1012.8136827987036
753.1920444208149

75

 27%|██▋       | 476/1747 [00:00<00:01, 1195.80it/s]

673.8099838473923
657.1972629801544
728.8327873172561
751.7257879636863
738.4722834983337

657.1972629801544
1239.9636677483186

670.3795814471918
629.0361195410934
696.8441844206413
683.2518752834546
633.102157912498

633.102157912498
1116.1240536816063

608.6790346966807
461.87646702033896
732.9768329000055
740.2446481825211
751.5712164658523

461.87646702033896
1239.9636677483186

736.3031699650213
810.4799664214424
691.2733098156325
550.6979090664759
744.4143840810281

691.2733098156325
1271.5463307512155

718.6353315437888
593.7169465603581
773.0049978410835
759.6908811730679
759.632715893738

593.7169465603581
1239.9636677483186

682.7374306154168
696.2584889460346
812.9496643329813
767.950419106024
861.3727691875062

682.7374306154168
1127.6519556049934

709.1561670123515
823.0838275021935
681.9298174373164
683.3858753704399
761.4907487430269

709.1561670123515
1127.6519556049934

693.3011319403817
780.6302931638909
795.0033246929133
794.2318219828443
776.1562305185626

693.3011

 39%|███▉      | 689/1747 [00:00<00:00, 1127.84it/s]

...

793.6248550953513
761.4598865870574
815.0510978860502

600.7163126080902
1239.9636677483186

1075.4377350031266
1095.660625287846
970.2386858481315
1042.7644439861467
980.2642205561317

970.2386858481315
1271.5463307512155

1007.6952733329856
1073.5846730960654
1016.0668230305816
1033.8504310637743
1001.9705834041052

1016.0668230305816
1271.5463307512155

1070.3306763104865
1056.93804039294
967.8775193640993
1017.9589997039947
947.9620363725805

1070.3306763104865
1127.6519556049934

915.9620325412297
893.5867220512966
942.0318926612414
925.4601115316121
914.7377021905895

942.0318926612414
1271.5463307512155

731.0483212179696
744.7449395748424
735.3448285279051
718.3520546181602
753.1847146441149

744.7449395748424
1239.9636677483186

609.9448701103806
643.6254169632732
755.369155369866
671.6307260099192
759.5639192143623

759.5639192143623
1116.1240536816063

716.2144802864827
617.6725540782321
795.0101500008614
779.0352618225087
812.3376725061962

812.3376725061962
1116.1240536816

 90%|████████▉ | 1568/1747 [00:01<00:00, 1041.15it/s]

1022.3911200391766
1271.5463307512155

622.6785258111413
691.6857883651949
673.1410123757249
636.384157173081
707.2459529484729

622.6785258111413
1127.6519556049934

663.5667626084834
709.8413310647827
770.7591330597813
769.2821153294034
774.5873948047115

663.5667626084834
1127.6519556049934

757.7420536939255
659.660812731963
801.4532032813819
833.5592992463389
810.0951752995135

801.4532032813819
1271.5463307512155

1003.4825244821749
1054.2806492935965
1020.3588435900749
1031.1788500803366
1009.4642303921873

1020.3588435900749
1271.5463307512155

640.0302420648136
595.3697039997863
623.869577719058
654.7544715663299
658.1895445894838

623.869577719058
1271.5463307512155

880.9894487592782
877.1486725006301
813.2430273331254
901.776269687396
839.8286882846305

880.9894487592782
1127.6519556049934

678.3517806372854
672.9218980691372
625.7882314530351
600.1207610933477
635.2167028211098

678.3517806372854
1127.6519556049934

1110.2206816508478
1091.4830449492636
1002.2578480834194


100%|██████████| 1747/1747 [00:01<00:00, 1065.84it/s]

877.4749184430711
846.9990975750087
817.2124591385226
874.7647819099183
819.7977570283686

817.2124591385226
1271.5463307512155

601.4474454165747
583.2810253031723
573.4880538693172
588.6751905830031
588.319003182186

573.4880538693172
1271.5463307512155

578.0392127965807
518.5216925348084
484.8632185234625
542.7794286604357
517.9329281018748

484.8632185234625
1271.5463307512155

1048.641709734657
996.5809545537944
1035.3711750916236
1086.4872514730987
1059.095993540191

996.5809545537944
1239.9636677483186

757.4561246738012
829.8554165516488
879.6338020170804
828.8552931057045
910.3367948842266

829.8554165516488
1239.9636677483186

746.5631450842745
676.3645249869097
766.2079959179082
774.0374547890984
809.5916228592665

766.2079959179082
1271.5463307512155

678.7618741296808
717.8317795330214
561.4406835945055
620.1279976981057
637.0656833388266

561.4406835945055
1271.5463307512155

715.2046973186926
636.1603002995415
621.3504211459731
743.0815063493086
573.9704632358165

621.3

In [ ]:
evaluate(updated_predictions, y_test)

Overall accuracy: 0.48311390955924444
Accuracy for class 2: 0.9359605911330049
Accuracy for class 4: 0.9651162790697675
Accuracy for class 0: 0.9856459330143541
Accuracy for class 3: 0.8947368421052632
Accuracy for class 1: 0.972972972972973
Accuracy for class -1: 0.006968641114982578


In [ ]:
macro_f1_score = f1_score(updated_predictions, y_test, average='macro')
print("Macro F1 score: {}".format(macro_f1_score))

micro_f1_score = f1_score(updated_predictions, y_test, average='micro')
print("Micro F1 score: {}".format(micro_f1_score))

weighted_f1_score = f1_score(updated_predictions, y_test, average='weighted')
print("Weighted F1 score: {}".format(weighted_f1_score))

norm_f1_score = f1_score(updated_predictions, y_test, average=None)
print("F1 score for each class: {}".format(norm_f1_score))

Macro F1 score: 0.5408110073556099
Micro F1 score: 0.48311390955924444
Weighted F1 score: 0.63235153942601
F1 score for each class: [0.01384083 0.66237942 0.6728972  0.74074074 0.60895522 0.54605263]


## Cosine Similarity

In [ ]:
def cos_similarity(centroid, data_point):
    centroid = centroid.reshape(1, -1)
    data_point = data_point.reshape(1, -1)
    return LA.norm(cosine_similarity(centroid, data_point))
    
def generate_centroids(features, expected):
    result_dict = {}
    for i in range(len(features)):
        c = expected[i]
        f = features[i]
        print
        if c not in result_dict:
            result_dict[c] = [np.array(f)]
        else:
            result_dict[c].append(np.array(f))
    for k, v in result_dict.items():
        result_dict[k] = [v, np.mean(np.array(v), axis=0)]
    return result_dict

def generate_thresholds_sim(centroids):
    result_dict = {}
    for k, [points, centroid] in centroids.items():
        current_min = 1
        for point in points:
            current_sim = cos_similarity(centroid, point)
            if current_min > current_sim:
                current_min = current_sim
        result_dict[k] = current_min
    return result_dict

In [ ]:
def check_negative_sim(threshold, centroid, feature):
    similarity = cos_similarity(centroid, feature)
    print(similarity)
    print(threshold)
    print()
    if similarity < threshold:
        return True
    return False

### Base Model

In [ ]:
extracted_features = feature_extractor.predict(x = trainval_loader)

In [ ]:
centroids_sim_dict = generate_centroids(extracted_features, y_trainval)
thresholds_sim_dict = generate_thresholds_sim(centroids_sim_dict)
print(thresholds_sim_dict)

{4: 0.6452441, 0: 0.5903251, 3: 0.607296, 1: 0.5934234, 2: 0.59954524}


In [ ]:
test_features = feature_extractor.predict(x = test_loader)

In [ ]:
test_classes = model.predict_classes(x=test_loader, batch_size=None)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
updated_predictions = []

for i in range(len(test_features)):
    predicted_class = test_classes[i]
    threshold = thresholds_sim_dict[predicted_class]
    centroid = centroids_sim_dict[predicted_class][1]
    feature = test_features[i]
    for c, thresh in thresholds_sim_dict.items():
        print(cos_similarity(centroids_sim_dict[c][1], feature))
    print()
    check = check_negative_sim(threshold, centroid, feature)
    if check:
        updated_predictions.append(-1)
    else:
        updated_predictions.append(predicted_class)

Streaming output truncated to the last 5000 lines.
0.5296407

0.7806202
0.607296

0.7528894
0.847187
0.5786557
0.3764298
0.3668459

0.847187
0.5903251

0.9076324
0.4917856
0.61866164
0.42958468
0.37093684

0.9076324
0.6452441

0.37411806
0.38554868
0.44314006
0.72389853
0.845824

0.845824
0.59954524

0.7842076
0.7216675
0.5336968
0.5228219
0.35033852

0.7842076
0.6452441

0.6662873
0.7106899
0.52480716
0.5762829
0.59819704

0.7106899
0.5903251

0.6431348
0.59105843
0.5886396
0.79091716
0.6116166

0.79091716
0.5934234

0.8422859
0.354815
0.66268384
0.58199716
0.6411814

0.8422859
0.6452441

0.55645174
0.3964571
0.5157601
0.88007766
0.7301694

0.88007766
0.5934234

0.5038367
0.37297472
0.47213423
0.671047
0.88231516

0.88231516
0.59954524

0.84585524
0.44010764
0.7441691
0.41348484
0.64193094

0.7441691
0.607296

0.81244373
0.5391653
0.7375891
0.40986514
0.514054

0.7375891
0.607296

0.43879053
0.4628412
0.5577142
0.80240864
0.82462656

0.82462656
0.59954524

0.5141313
0.7315118
0.721981

In [ ]:
evaluate(updated_predictions, y_test)

Overall accuracy: 0.48082427017744706
Accuracy for class 2: 0.9359605911330049
Accuracy for class 4: 0.9651162790697675
Accuracy for class 0: 0.9856459330143541
Accuracy for class 3: 0.8947368421052632
Accuracy for class 1: 0.972972972972973
Accuracy for class -1: 0.0023228803716608595


In [ ]:
macro_f1_score = f1_score(updated_predictions, y_test, average='macro')
print("Macro F1 score: {}".format(macro_f1_score))

micro_f1_score = f1_score(updated_predictions, y_test, average='micro')
print("Micro F1 score: {}".format(micro_f1_score))

weighted_f1_score = f1_score(updated_predictions, y_test, average='weighted')
print("Weighted F1 score: {}".format(weighted_f1_score))

norm_f1_score = f1_score(updated_predictions, y_test, average=None)
print("F1 score for each class: {}".format(norm_f1_score))

Macro F1 score: 0.5382164978576821
Micro F1 score: 0.48082427017744706
Weighted F1 score: 0.6327856058642363
F1 score for each class: [0.00463499 0.66237942 0.66976744 0.73929961 0.60895522 0.5442623 ]


### Vggish Model

In [ ]:
vggish_trainval_features = None
for data, labels in tqdm(vggish_trainval_loader):
  if vggish_trainval_features is None: vggish_trainval_features = data
  else: vggish_trainval_features = np.concatenate((vggish_trainval_features, data), axis=0)

100%|██████████| 222/222 [40:06<00:00, 10.84s/it]


In [ ]:
vggish_centroids_sim_dict = generate_centroids(vggish_trainval_features, y_trainval)
vggish_thresholds_sim_dict = generate_thresholds_sim(vggish_centroids_sim_dict)
print(vggish_thresholds_sim_dict)

{4: 0.779731590950885, 0: 0.7462281809032909, 3: 0.7593273519904803, 1: 0.7803307604456015, 2: 0.7867504675040624}


In [ ]:
test_classes = model.predict_classes(x=test_loader, batch_size=None)

In [ ]:
vggish_test_features = None
for data, labels in tqdm(vggish_test_loader):
  if vggish_test_features is None: vggish_test_features = data
  else: vggish_test_features = np.concatenate((vggish_test_features, data), axis=0)

100%|██████████| 110/110 [19:50<00:00, 10.82s/it]


In [ ]:
updated_predictions = []

for i in tqdm(range(len(vggish_test_features))):
    predicted_class = test_classes[i]
    threshold = vggish_thresholds_sim_dict[predicted_class]
    centroid = vggish_centroids_sim_dict[predicted_class][1]
    feature = vggish_test_features[i]
    for c, thresh in vggish_thresholds_sim_dict.items():
        print(cos_similarity(vggish_centroids_sim_dict[c][1], feature))
    print()
    check = check_negative_sim(threshold, centroid, feature)
    if check:
        updated_predictions.append(-1)
    else:
        updated_predictions.append(predicted_class)

  4%|▍         | 73/1747 [00:00<00:04, 361.43it/s]

0.7851245572579199
0.7602094239258231
0.8703186423304896
0.7973615133555609
0.8884608480103453

0.8884608480103453
0.7867504675040624

0.8814402896706768
0.8471728053213162
0.8438087856662726
0.8407432719466528
0.8265842112476399

0.8814402896706768
0.779731590950885

0.8853949457442012
0.8563919828272295
0.8595814774273012
0.8527813094839409
0.8444105499667454

0.8853949457442012
0.779731590950885

0.8132379185198653
0.821188535285692
0.8624629153885393
0.8174141577883105
0.8935495105856925

0.8935495105856925
0.7867504675040624

0.9031848364325744
0.9147195377831752
0.9105431385183712
0.887583197198728
0.9174035492114221

0.9031848364325744
0.779731590950885

0.8577658651435444
0.8261796894850614
0.8384006062861222
0.8644224594927836
0.8156338290372265

0.8577658651435444
0.779731590950885

0.836439113775278
0.8805374496908045
0.7861208245614002
0.8151585120949778
0.7856096270304576

0.8805374496908045
0.7462281809032909

0.7845284612392125
0.8068480913906129
0.8609255037148906
0.793

  8%|▊         | 142/1747 [00:00<00:04, 349.21it/s]


0.8887795209114556
0.9214512248248984
0.912767203388086
0.905676234206298

0.9286257474020524
0.779731590950885

0.7935604463330718
0.8082839580803756
0.8450503561200342
0.7860121405809435
0.8268872094063668

0.8450503561200342
0.7593273519904803

0.8625226380647557
0.8157797595423669
0.8471378570735298
0.9085477086344915
0.8305661531454953

0.9085477086344915
0.7803307604456015

0.8564358770886044
0.8647823607504797
0.8527236661171355
0.8388466000714551
0.8633451324648949

0.8647823607504797
0.7462281809032909

0.8799472270121722
0.8625647862538556
0.9239802280183232
0.871929892702302
0.8980396300594602

0.9239802280183232
0.7593273519904803

0.9332009663035505
0.8886357589995411
0.8665432102249251
0.8662379353902048
0.8578611988835937

0.9332009663035505
0.779731590950885

0.8903417018403581
0.9226971633781336
0.8693203379040062
0.8824639477889049
0.8563233049610519

0.8903417018403581
0.779731590950885

0.8834022190452743
0.859284240174675
0.8579732974323562
0.8645015889044949
0.83

 12%|█▏        | 205/1747 [00:00<00:04, 327.37it/s]

...


0.8881773864394451
0.9001285044209253

0.9091744382222227
0.7593273519904803

0.918411699655766
0.8959672782804928
0.8715805813403994
0.8603064928730971
0.8680037179595046

0.918411699655766
0.779731590950885

0.8372813055758935
0.8035817926754218
0.819785852169934
0.8036985249385873
0.8207601330844811

0.8372813055758935
0.779731590950885

0.920446813780714
0.9396205441813489
0.8574015960953185
0.8831119050648453
0.8527169699490458

0.920446813780714
0.779731590950885

0.8482812936603799
0.9010633144463912
0.8450168665486812
0.830242943342999
0.8673905527853982

0.8450168665486812
0.7593273519904803

0.8148848969361877
0.8077698429865011
0.8486614548385829
0.8211121381352682
0.8640714983585707

0.8640714983585707
0.7867504675040624

0.7877940947644522
0.7675733430380982
0.8000351049549211
0.7872229364367702
0.7977689301158035

0.8000351049549211
0.7593273519904803

0.8997329548387766
0.9228982248807802
0.9079538449363935
0.8920836748418439
0.919584839836036

0.9079538449363935
0.7593

 96%|█████████▌| 1674/1747 [00:05<00:00, 302.97it/s]


0.8606323158428681
0.8906257867351064

0.902144249327316
0.7593273519904803

0.8984593825736791
0.942548698626039
0.8704045056556696
0.8805743849939971
0.8691488428510055

0.8704045056556696
0.7593273519904803

0.838012868980708
0.8532025373042637
0.8535939191571633
0.8321960954125238
0.8529689738091668

0.8532025373042637
0.7462281809032909

0.9180531310818063
0.9340071793532224
0.8564542709060885
0.8747494245135778
0.8525384930565214

0.9180531310818063
0.779731590950885

0.8241724525382781
0.8285138965812754
0.8592652353962342
0.9028068939834313
0.8620855191108103

0.9028068939834313
0.7803307604456015

0.8990781791008287
0.9415726172766332
0.8913551793445387
0.8879203269771485
0.8906834154216658

0.9415726172766332
0.7462281809032909

0.8538474646100482
0.8529089663101361
0.8848067825902427
0.8716061994271106
0.884616754867485

0.8848067825902427
0.7593273519904803

0.8225264352620195
0.8338663457310482
0.8515822977224987
0.8458620296609768
0.8460311775063858

0.8515822977224987
0

100%|██████████| 1747/1747 [00:05<00:00, 318.92it/s]



0.8997848327588935
0.7867504675040624

0.8691097305323605
0.8364073767719595
0.8152569899365953
0.8110824587471527
0.8119203809824649

0.8691097305323605
0.779731590950885

0.9052635237643016
0.9033257176452566
0.829312423664075
0.843629272343373
0.8309810510956193

0.9052635237643016
0.779731590950885

0.8718205629864049
0.865920385131223
0.8407701690010325
0.8480529512101194
0.8405523850030205

0.8718205629864049
0.779731590950885

0.8623538452788931
0.923881590586969
0.8331870739243509
0.810891780006902
0.8275222672136882

0.923881590586969
0.7462281809032909

0.8926673632089002
0.8863139584591437
0.931474206807546
0.911838588101498
0.922805742971744

0.922805742971744
0.7867504675040624

0.9049958319800152
0.922028739530541
0.8554115138330602
0.8886209994682819
0.8462634224612418

0.9049958319800152
0.779731590950885

0.8797609167289262
0.8635822443597371
0.8800452065179967
0.9290776336490254
0.8649069456247145

0.8797609167289262
0.779731590950885

0.8541709508076116
0.861430010

In [ ]:
evaluate(updated_predictions, y_test)

Overall accuracy: 0.48597595878649114
Accuracy for class 2: 0.9359605911330049
Accuracy for class 4: 0.9651162790697675
Accuracy for class 0: 0.9856459330143541
Accuracy for class 3: 0.8947368421052632
Accuracy for class 1: 0.972972972972973
Accuracy for class -1: 0.012775842044134728


In [ ]:
macro_f1_score = f1_score(updated_predictions, y_test, average='macro')
print("Macro F1 score: {}".format(macro_f1_score))

micro_f1_score = f1_score(updated_predictions, y_test, average='micro')
print("Micro F1 score: {}".format(micro_f1_score))

weighted_f1_score = f1_score(updated_predictions, y_test, average='weighted')
print("Weighted F1 score: {}".format(weighted_f1_score))

norm_f1_score = f1_score(updated_predictions, y_test, average=None)
print("F1 score for each class: {}".format(norm_f1_score))

Macro F1 score: 0.5432736573388565
Micro F1 score: 0.48597595878649114
Weighted F1 score: 0.6318731682194519
F1 score for each class: [0.02522936 0.66237942 0.66976744 0.74363992 0.60986547 0.54876033]
